In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

In [2]:
STORAGE_PATH_DATA = "../../persisted_data/feather/{}.feather"
STORAGE_PATH_MODELS = "../../persisted_data/models/{}.pth"
def load_data_set(name):
    data = pd.read_feather(STORAGE_PATH_DATA.format(name))
    return data

In [3]:
# define the used indicators
standard_indicators = ["sma10", "sma20", "sma50", "sma100", "lwma10", "lwma20", "lwma50", "lwma100", "lwma200",
                       "ema10", "ema20", "ema50", "ema100", "rate_of_change20", "rate_of_change50", 
                       "horizontal_position20", "horizontal_position50", "horizontal_position100",
                       "regression_position20", "regression_position50", "regression_position100",
                       "bollinger_position20_2", "bollinger_position50_2", "bollinger_position100_2"]

svm_indicators_f_classif_selected = ['macd20_50', 'macd_signal20_50', 'ma_trend50_200', 'macd50_200',
                                     'macd_signal50_200', 'aaron_up15', 'aaron_up25', 'aaron_up40',
                                     'bollinger_position20_2', 'bollinger_position50_2',
                                     'bollinger_position100_2', 'bollinger_position200_2', 'rsi14', 'rsi20',
                                     'rsi_logistic20', 'horizontal_lower20', 'horizontal_position20',
                                     'horizontal_threshold20', 'horizontal_lower50', 'horizontal_position50',
                                     'horizontal_threshold50', 'horizontal_lower100',
                                     'horizontal_position100', 'horizontal_threshold100',
                                     'horizontal_lower200', 'horizontal_position200',
                                     'horizontal_threshold200', 'regression_position200', 'chande50',
                                     'chande100']

In [4]:
class PriceHistoryDataset(Dataset):

    def __init__(self, file_name, selected_indicators=standard_indicators, future_indicator="future_price"):
        # Read the feather data set file
        self.data = load_data_set(file_name)
        self.indicators = torch.tensor(self.data[selected_indicators].values.astype(np.float32)) 
        self.future = torch.tensor(np.where(self.data[future_indicator] > 1, 1, 0).astype(np.float32)) 

    def __getitem__(self, i):
        return self.indicators[i], self.future[i]

    def __len__(self):
        return len(self.data)

In [5]:
# create a data loader for training
training_dataset = PriceHistoryDataset("all_stocks_20spy_20shift_normalized")
dataloader_training = DataLoader(training_dataset, 64, shuffle=True, num_workers=4)

In [6]:
class PriceHistoryNetwork(nn.Module):
    def __init__(self):
        super().__init__() 
        # define layers 
        self.fc1 = nn.Linear(24, 12)
        self.sigm1 = nn.Sigmoid()
        self.fc2 = nn.Linear(12, 8)
        self.sigm2 = nn.Sigmoid()
        self.fc3 = nn.Linear(8, 1)
        self.sigm3 = nn.Sigmoid()


    def forward(self, x):
        x = self.fc1(x)
        x = self.sigm1(x)
        x = self.fc2(x)
        x = self.sigm2(x)
        x = self.fc3(x)
        x = self.sigm3(x)

        return x

In [7]:
# training function
def train(net, train_loader, loss_function):
    optimizer = torch.optim.SGD(net.parameters(),lr=0.01, momentum=0.9)
    epoches = 10
    for epoch in range(epoches):
        net.train() 
        for i, (indicators, future) in enumerate(train_loader):
            outs = torch.flatten(net(indicators))
            loss = loss_function(outs, future)

            # clear grads
            optimizer.zero_grad()
            
            # backward
            loss.backward()
            
            # update parameters
            optimizer.step()
            
    return net

In [8]:
# create and train the network
net = PriceHistoryNetwork()
loss_function = nn.BCELoss()
train(net, dataloader_training, loss_function)

PriceHistoryNetwork(
  (fc1): Linear(in_features=24, out_features=12, bias=True)
  (sigm1): Sigmoid()
  (fc2): Linear(in_features=12, out_features=8, bias=True)
  (sigm2): Sigmoid()
  (fc3): Linear(in_features=8, out_features=1, bias=True)
  (sigm3): Sigmoid()
)

In [9]:
# Save the weight of current model to disk
torch.save(net.state_dict(), STORAGE_PATH_MODELS.format("neural_network_standard"))